In [ ]:

import dask.dataframe as dd
import pandas as pd
import numpy as np
import multiprocessing
from scipy.stats import mannwhitneyu 
from statsmodels.stats.multitest import multipletests

In [ ]:
useNewData = True

if useNewData == False:
    ccle_expression = pd.read_csv("sli-algo inputs/CCLE_Expression.csv")
else:
    ccle_expression = pd.read_csv("sli-algo inputs/OmicsExpressionProteinCodingGenesTPMLogp1BatchCorrected_24Q2.csv")

In [ ]:
if useNewData == False:
    gene_effect = pd.read_csv("sli-algo inputs/CRISPR_gene_effect.csv")
else:
    gene_effect = pd.read_csv("sli-algo inputs/CRISPRGeneEffect_24Q2.csv")

gene_effect.rename(columns={gene_effect.columns[0]:"DepMap_ID"}, inplace=True)

In [ ]:
# Rename the first column of the DataFrame (since Dask is lazy, we need to get the list of column names first)
column_names = list(ccle_expression.columns)
ccle_expression = ccle_expression.rename(columns={column_names[0]: "DepMap_ID"})

# Convert expression data to long format
expr_long = ccle_expression.melt(id_vars="DepMap_ID", var_name="gene_name", value_name="gene_expression")

expr_long["gene_name"] = expr_long["gene_name"].str.replace(r" \(.*", "", regex=True)

# Convert CRISPR KO data to long format
crispr_long = gene_effect.melt(id_vars="DepMap_ID", var_name="gene_name", value_name="crispr_effect")

crispr_long["gene_name"] = crispr_long["gene_name"].str.replace(r" \(.*", "", regex=True)

final_table_pd = expr_long.merge(crispr_long, on=["DepMap_ID", "gene_name"], how="outer")

#REVERSE SIGN of gene effect to make the biggest effect positive
final_table_pd["crispr_effect"] = -final_table_pd["crispr_effect"]

In [ ]:
# Preprocess data into a dictionary for quick access
gene_groups = {gene: group for gene, group in final_table_pd.groupby('gene_name')}

final_table = final_table_pd

In [ ]:
def split_populations(mutant_data, lower_percentile, upper_percentile):
    gene_expr = mutant_data['gene_expression'].values
    low_bound = np.percentile(gene_expr, lower_percentile)
    high_bound = np.percentile(gene_expr, upper_percentile)
    
    mask_low = gene_expr <= low_bound
    mask_high = gene_expr > high_bound
    
    population = np.where(mask_low, 'low', np.where(mask_high, 'high', None))
    mutant_data = mutant_data.assign(population=population)
    return mutant_data.dropna(subset=['population'])

def one_sided_test(mutant, data, gene):
    data = data.drop_duplicates(subset=["DepMap_ID", "gene_name", "crispr_effect", "population"])
    if data.empty or data["population"].nunique() < 2:
        return None
    
    counts = data["population"].value_counts()
    high_count = counts.get("high", 0)
    low_count = counts.get("low", 0)
    if high_count == 0 or low_count == 0:
        return None
    
    try:
        stat, p_value = mannwhitneyu(
            data[data["population"] == "low"]["crispr_effect"],
            data[data["population"] == "high"]["crispr_effect"],
            alternative='less',
            method='asymptotic'
        )
    except ValueError:
        return None
    
    result = {
        "mutant": mutant,
        "gene": gene,
        "high": high_count,
        "low": low_count,
        "p_value": p_value,
        "statistic": stat,
    }

    return result

def run_hypothesis_test_unique_percentiles(mutant, gene, low_percentile, high_percentile):
    # Fetch CRISPR data
    crispr_data = gene_groups[gene].dropna(subset=['crispr_effect'])
    if crispr_data.empty:
        return create_empty_result(mutant, gene, low_percentile, high_percentile)
    
    # Fetch mutant data 
    mutant_data = gene_groups.get(mutant, pd.DataFrame())
    # Filter it to eliminate entries that aren't in CRISPR_data 
    mutant_data = mutant_data[mutant_data['DepMap_ID'].isin(crispr_data['DepMap_ID'])].dropna(subset=['gene_expression'])
    if mutant_data.empty:
        return create_empty_result(mutant, gene, low_percentile, high_percentile)
    
    # 3. Split populations
    expression_populations = split_populations(mutant_data, low_percentile, high_percentile)
    if expression_populations.empty:
        return create_empty_result(mutant, gene, low_percentile, high_percentile)
    
    # 4. Map population to CRISPR data
    population_map = expression_populations.set_index('DepMap_ID')['population']
    crispr_population_data = crispr_data[crispr_data['DepMap_ID'].isin(population_map.index)].copy()
    crispr_population_data['population'] = crispr_population_data['DepMap_ID'].map(population_map)
    crispr_population_data.dropna(subset=['population'], inplace=True)
    
    # 5. Compute mean/median differences
    mean_median = crispr_population_data.groupby('population')['crispr_effect'].agg(['mean', 'median']).diff().fillna(0)
    diff_mean = mean_median['mean'].iloc[-1]
    diff_median = mean_median['median'].iloc[-1]
    
    # 6. Perform test
    result = one_sided_test(mutant, crispr_population_data, gene)
    if result is None:
        return create_empty_result(mutant, gene, low_percentile, high_percentile)
    
    result['low_percentile'] = low_percentile
    result['high_percentile'] = high_percentile
    result['diff_mean'] = diff_mean
    result['diff_median'] = diff_median
    return result

def create_empty_result(mutant, gene, low, high):
    result = {
        "mutant": [mutant],
        "gene": [gene],
        "high": [0],
        "low": [0],
        "p_value": [np.nan],
        "statistic": [np.nan],
        "low_percentile": [low],
        "high_percentile": [high],
        "diff_mean": [0],
        "diff_median": [0]
    }

    return result

In [ ]:
def run_crispr_database(mutants_to_include, ko_genes_to_include, low_percentile, high_percentile, threads=1):  
    # Create a list of all SL pairs to be tested (excluding self-pairs)
    mutant_gene_pairs = [(mutant, ko_gene, low_percentile, high_percentile)
             for mutant in mutants_to_include
             for ko_gene in ko_genes_to_include if ko_gene != mutant]
    
    # Convert the list of tuples into a DataFrame
    mutant_gene_pairs_df = pd.DataFrame(mutant_gene_pairs, columns=["mutant", "gene", "low_percentile", "high_percentile"])

    # Single-threaded execution
    if threads == 1:
        results = mutant_gene_pairs_df.apply(
            lambda row: run_hypothesis_test_unique_percentiles(
                row["mutant"], row["gene"], low_percentile, high_percentile
            ), axis=1
        )
        results_df = results.apply(pd.Series)
    
    elif threads > 1:
        # Create a function for multiprocessing
        def process_row(row):
            return run_hypothesis_test_unique_percentiles(
                row["mutant"], row["gene"], row["low_percentile"], row["high_percentile"]
            )

        # Convert DataFrame to list of dictionaries for multiprocessing
        rows = mutant_gene_pairs_df.to_dict('records')
    
        # Create a pool of workers
        with multiprocessing.Pool(processes=threads) as pool:
            # Map the function to the rows
            results_list = pool.map(process_row, rows)
        
        # Convert the list of results to a DataFrame
        results_df = pd.DataFrame(results_list)
    
    else:
        raise ValueError("Invalid number of threads")

    # Apply Benjamini-Hochberg correction to p-values
    for mutant in mutants_to_include: 
        p_values = results_df.loc[results_df['mutant'] == mutant, 'p_value'].values
        reject, corrected_p_values, _, _ = multipletests(p_values, method='fdr_bh')
        results_df.loc[results_df['mutant'] == mutant, 'adjusted_p_value'] = corrected_p_values

    # Format adjusted p-values in scientific notation
    results_df['adjusted_p_value'] = results_df['adjusted_p_value'].apply(lambda x: f"{x:.6E}")

    return results_df

In [ ]:
# Running the function for gene pairs that do not concern on of our "mutants", to see if expanding our criteria might be worth it

low_percentile = 10
high_percentile = 90
pairs_to_test = pd.read_csv("sli-algo inputs/SLKB_known_pairs_4_cell_lines.csv")

# Apply function row-wise
results = pairs_to_test.apply(lambda row: run_hypothesis_test_unique_percentiles(row['mutant'], row['gene'], low_percentile, high_percentile), axis=1)

# Convert the list of dicts to a DataFrame
results_df = pd.DataFrame(results.tolist())

results_df.sort_values(by="p_value", ascending=True)

,mutant,gene,high,low,p_value,statistic,low_percentile,high_percentile,diff_mean,diff_median
161,SOX10,SOX9,106,317,0.0,8554.0,10,90,-0.207059,-0.303324
242,WDR5,WDR83,107,107,0.000006,3733.0,10,90,-0.135551,-0.168416
151,EGFR,ERBB3,107,142,0.000011,5210.0,10,90,-0.117552,-0.060873
252,EGFR,ERBB2,107,142,0.000049,5404.0,10,90,-0.12826,-0.059449
174,BCL2L1,MAPK1,107,107,0.000051,3965.0,10,90,-0.106931,-0.101282
...,...,...,...,...,...,...,...,...,...,...
228,PAK1,PAK2,107,107,1.0,8747.0,10,90,0.249363,0.185014
112,SMARCA2,SMARCA4,107,107,1.0,9078.0,10,90,0.342753,0.3966
60,TBL1X,TBL1XR1,107,107,1.0,9853.0,10,90,0.468053,0.475338
277,TUBB,TUBB4B,107,107,1.0,9611.0,10,90,0.477246,0.392139


In [ ]:
low_percentile = 10
high_percentile = 90
pairs_to_test = pd.read_csv("sli-algo inputs/SLKB_known_pairs_10_cell_lines.csv")

# Apply function row-wise
results = pairs_to_test.apply(lambda row: run_hypothesis_test_unique_percentiles(row['mutant'], row['gene'], low_percentile, high_percentile), axis=1)

# Convert the list of dicts to a DataFrame
results_df = pd.DataFrame(results.tolist())

results_df.sort_values(by="p_value", ascending=True)


,mutant,gene,high,low,p_value,statistic,low_percentile,high_percentile,diff_mean,diff_median
10,PIP5K1A,PIP5K1C,107,107,0.225753,5383.0,10,90,-0.012489,-0.009447
6,MAPK1,MAPK3,107,107,0.513210,5739.0,10,90,0.006512,-0.002097
4,RPS6KB1,SGK2,107,107,0.914833,6345.0,10,90,0.015519,0.019512
5,MAP2K1,MAP2K2,107,107,0.924694,6375.0,10,90,0.041290,0.032665
3,CSNK2A1,CSNK2A2,107,107,0.940217,6429.0,10,90,0.021147,0.053402
9,MARK2,MARK3,107,107,0.961146,6523.0,10,90,0.037819,0.036312
1,UBE2A,UBE2B,107,107,0.978436,6640.0,10,90,0.026012,0.036756
8,HDAC1,HDAC2,107,107,0.998387,7058.0,10,90,0.135405,0.050214
7,HK1,HK2,107,107,0.999095,7137.0,10,90,0.116834,0.070866
2,CDK4,CDK6,107,107,1.000000,8184.0,10,90,0.346574,0.473508


In [ ]:
results_df["p_value"] = results_df["p_value"].apply(lambda x: x[0] if isinstance(x, list) else x)

results_df[results_df["p_value"] <= 0.01]

,mutant,gene,high,low,p_value,statistic,low_percentile,high_percentile,diff_mean,diff_median
15,KCNA2,KCNA7,107,305,5.378924e-03,13614.0,10,90,-0.035309,-0.032311
21,KCNB2,KCNG1,106,345,3.090479e-03,15071.0,10,90,-0.044241,-0.048488
83,PAK3,PAK4,107,151,6.899118e-04,6189.0,10,90,-0.051155,-0.047305
87,LATS1,LATS2,107,107,1.427520e-03,4373.0,10,90,-0.077099,-0.07932
98,PRKCA,PRKCE,107,107,2.610711e-03,4459.0,10,90,-0.065794,-0.058306
105,MMP2,MMP9,107,110,9.887706e-04,4454.0,10,90,-0.03258,-0.034505
151,EGFR,ERBB3,107,142,1.108874e-05,5210.0,10,90,-0.117552,-0.060873
161,SOX10,SOX9,106,317,1.891599e-14,8554.0,10,90,-0.207059,-0.303324
174,BCL2L1,MAPK1,107,107,5.142469e-05,3965.0,10,90,-0.106931,-0.101282
242,WDR5,WDR83,107,107,5.512890e-06,3733.0,10,90,-0.135551,-0.168416


In [24]:
# this is the list of KO genes that are to be tested for each mutant, it contains 10053 rows

ko_genes_to_include = pd.read_excel('sli-algo inputs/reactome genes list (to include).xlsx')['gene_name'].to_list()

# We make sure that all of the genes are incuded in our dataset, otherwise, we drop it and print it out
ko_genes_to_include = [gene for gene in ko_genes_to_include if gene in gene_groups or print("Removed:", gene)]

# And this is the list of mutant genes that are to be tested

# mutants_to_include = ["ARID1A","ARID1B","ARID2","BAP1","CREBBP","EED","KMT2C","KMT2D","PBRM1","SETD2","SMARCA2","SMARCA4","SMARCB1"]

mutants_to_include = ["ARID1A"]

Removed: SFTA3


In [1]:
results_df = run_crispr_database(mutants_to_include, ko_genes_to_include, 10, 90, threads = 1)

results_df

NameError: name 'run_crispr_database' is not defined